In [1]:
import numpy as np
import pandas as pd
from __future__ import print_function

train = pd.read_csv('train.csv')
train.shape # three columns are id, text, author

(19579, 3)

In [2]:
from keras.utils import to_categorical

a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
labels = np.array([a2c[a] for a in train.author])
labels = to_categorical(labels)
print(labels)

Using TensorFlow backend.


NameError: name 'df' is not defined

In [ ]:
text = train["text"]
print("text shape: ", text.shape)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters="", lower=True, split=" ", char_level=False)
tokenizer.fit_on_texts(text);
docs = tokenizer.texts_to_sequences(text);
print("text: ", text[0])
print("tokenized: ", docs[0])

In [36]:
from keras.preprocessing.sequence import pad_sequences

maxlen = np.amax([len(x) for x in docs], axis=0)
print("max doc length: ", maxlen)

docs = pad_sequences(sequences=docs, maxlen=maxlen)
print("docs[0]: ", len(docs[0]))

max doc length:  861
docs[0]:  861


In [51]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size=0.2)
input_dim = np.max(docs) + 1
embedding_dims = 20

In [52]:
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.models import Sequential

def create_model(input_dim, embedding_dims=20, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [53]:
model = create_model(input_dim, embedding_dims=embedding_dims)

In [55]:
from keras.callbacks import EarlyStopping

model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=2,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/2
15663/15663 [==============================] - 48s 3ms/step - loss: 1.0779 - acc: 0.4059 - val_loss: 1.0764 - val_acc: 0.3950
Epoch 2/2
15663/15663 [==============================] - 47s 3ms/step - loss: 1.0584 - acc: 0.4243 - val_loss: 1.0497 - val_acc: 0.4254
